# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.4,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.3,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.3,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.3,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="indy0_tcp", dims=(0.1,0.07,0.02), 
                   center=(0.0,0.0,0.14), rpy=(0,np.pi,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="indy0_tcp", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [11]:
# gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
#                  dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
# pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=FixtureSlot, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepFramer, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
# brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
#                              action_points_dict = {"handle": FixturePoint("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
#                                                    "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [17]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepFrame("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepFrame("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepFrame("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepFrame("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepFrame("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepFrame("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [19]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [20]:
mplan.motion_filters = checkers_all

In [21]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 'floor', 0, 0, 0)


In [23]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [24]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler

In [25]:
tplan.new_node_sampler = UniformNodeSampler(3)
tplan.parent_node_sampler = UniformNodeSampler(3)

In [26]:
class BindingSampler:
    def __init__(self, gamma_decay=0.8):
        self.gamma_decay = gamma_decay

    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [self.prob_dict[node] for node in nodes]
        probs = np.divide(probs, np.sum(probs))
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.prob_dict[node] = self.prob_dict[node] * self.gamma_decay
        return node

# CustomRule

In [27]:
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [28]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        
    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.call_count_dict = multiprocess_manager.dict()
            self.count_lock = multiprocess_manager.Lock()
        else:
            self.call_count_dict = dict()
            self.count_lock = DummyBlock()
            
        no_enter_sidxes = [stype==SweepLineTask for stype in enumerate(self.pscene.subject_type_list)]
        no_enter_initials = [tplan.initial_state.node[sidx] for sidx in no_enter_sidxes]
            
        self.node_dict={}
        for node, leafs in tplan.node_dict.items():
            self.node_dict[node] = leafs
            if not all([node[k] in terms for k, terms in tplan.unstoppable_terminals.items()]):
                tplan.node_dict[node] = leafs = set([]) ## unstoppable node change will be reserved by this custom rule

        self.node_parent_dict={}
        for node, parents in tplan.node_parent_dict.items():
            self.node_parent_dict[node] = parents
            tplan.node_parent_dict[node] = parents = set(
                [parent for parent in parents ## unstoppable node change will be reserved by this custom rule
                 if all([parent[k] in terms for k, terms in tplan.unstoppable_terminals.items()])])
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
        print("CustomRule call")
        if snode_src is not None:
            with self.count_lock:
                if snode_src.idx in self.call_count_dict:
                    self.call_count_dict[snode_src.idx] = self.call_count_dict[snode_src.idx] + [snode_new.state.node]
                else:
                    self.call_count_dict[snode_src.idx] = [snode_new.state.node]
            
            node_src = snode_src.state.node
            node_new = snode_new.state.node
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(node_src, node_new)])[0]
            print("{}->{} , diff: {}".format(node_src, node_new, diff_sidxes))
            if len(diff_sidxes)==0:
                return False, None
            diff_sidx = diff_sidxes[0]
            diff_sname = self.pscene.subject_name_list[diff_sidx]
            diff_subject = self.pscene.subject_dict[diff_sname]
            
            if isinstance(diff_subject, SweepLineTask):
                print("Rule for SweepLineTask")
                with tplan.snode_dict_lock:
                    if node_src[diff_sidx] not in tplan.unstoppable_terminals[diff_sidx]:
                        snode_list = tplan.node_snode_dict[node_src]
                        if snode_src.idx in snode_list:
                            print("Removing {} from {}: {} not in {}".format(snode_src.idx, snode_list, node_src[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            print(snode_list)
                            snode_list.remove(snode_src.idx)
                            tplan.node_snode_dict[node_src] = snode_list
                            print(tplan.node_snode_dict[node_src])
                        if len(snode_list) == 0 and node_new in tplan.neighbor_nodes:
                            del tplan.neighbor_nodes[node_new]
                    elif connection_result:
                        if node_new[diff_sidx] in tplan.unstoppable_terminals[diff_sidx]: ## in terminal condition
                            print("Check home: {} in {}".format(node_new[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            link_name = self.pscene.gscene.NAME_DICT[snode_new.state.binding_state[diff_sidx][-1]].link_name
                            rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                            link_name in chain_vals['link_names']]
                            if len(rname_candis)==0:
                                print("no robot candis")
                            else:
                                print("reserve homing = {}".format(snode_new.idx))
                                newstate = snode_new.state.copy(self.pscene)
                                jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                                newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                                return True, newstate
                        else:
                            print("Reserve next wp from {}".format(snode_new.idx))
                            snode_list = tplan.node_snode_dict[node_new]
                            if snode_new.idx in snode_list:
                                print("Removing {} from {} as next wp is reserved".format(snode_new.idx, snode_list))
                                snode_list.remove(snode_new.idx)
                                tplan.node_snode_dict[node_new] = snode_list
                            next_node_candis = list(self.node_dict[node_new])
                            assert len(next_node_candis) == 1, "non-terminal sweep task should have only 1 leaf {}-{}".format(node_new, next_node_candis)
                            return True, next_node_candis[0] 
                return False, None
            
            if tplan.reserved_attempt:
                    return False, None
            elif connection_result:
                if isinstance(diff_subject, AbstractObject):
                    print("Rule for AbstractObject")
                    link_name1 = snode_src.state.state_param[diff_sname][0]
                    link_name2 = snode_new.state.state_param[diff_sname][0]
                    rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                   link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                    if len(rname_candis)==0:
                        print("no robot candis")
                        return False, None
                    else:
                        print("try homing")
                        newstate = snode_new.state.copy(self.pscene)
                        jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                        newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                        return True, newstate
        return False, None

In [29]:
tplan.custom_rule = CustomRule(pscene)

## MUST CHECK! no more than 1 trial from non-terminal nodes
## MUST CHECK! no more than 1 trial to same framed motion
## MUST CHECK! efficient binding

In [ ]:
for k, v in tplan.custom_rule.call_count_dict.items():
    print("{}: {} - {}".format(k,tplan.snode_dict[k].state.node, v))

In [ ]:
gtimer.reset()
goal_nodes = [("floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
CustomRule call
=============== try extend to goal ('floor', 'floor', 0, 0, 0) -> ('floor', 'floor', 0, 1, 0) =================
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0)
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) , diff: [3]
Rule for SweepLineTask
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: []
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = fail
transition motion tr

try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
try: 8 - ('floor', 'floor', 0, 0, 1)->('floor', 'floor', 0, 0, 2)
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
Removing 6 from [6, 7] as next wp is reserved
CustomRule call
Rule for SweepLineTask
ERROR sampling parent from : ('grip1', 'floor', 0, 1, 0) / parent nodes: set([('grip1', 'floor', 0, 0, 0)])
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = success
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: []
try: 6 - ('floor', 'floor', 0, 0, 1)->('floor', 'floor', 0, 0, 2)
branching: 0->6 (0.72/100.0 s, steps/err: 12(231.830120087 ms)/0.00184519655873)
'a' must be greater than 0 unless no samples are taken
CustomRule call
Reserve next wp from 5
ERROR sampling parent from : ('grip1', 'floor', 0, 0, 1) / parent nodes: set([('grip1', 'floor', 0, 0, 0)])
'a' must be greater than 0 unless no samples are taken
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor

try: 13 - ('floor', 'floor', 0, 0, 1)->('floor', 'floor', 0, 0, 2)
try: 14 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0)
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) , diff: [4]
joint motion tried: True
Rule for SweepLineTask
Reserve next wp from 15
transition motion tried: False
CustomRule call
('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: []
Removing 15 from [15] as next wp is reserved
CustomRule call
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = success
result: 12 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = success
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
branching: 0->15 (1.13/100.0 s, steps/err: 12(66.7500495911 ms)/0.00170259594099)
branching: 12->16 (1.14/100.0 s, steps/err: 34(182.564973831 ms)/8.2049748031e-16)
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
try: 15 - ('floor', 'floor', 0, 0, 1)->('floor', 'floor', 0, 0, 

branching: 19->24 (3.91/100.0 s, steps/err: 29(1143.93615723 ms)/9.26360840719e-16)
branching: 22->23 (3.91/100.0 s, steps/err: 13(82.6010704041 ms)/0.00165190990889)
try: 24 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
try: 23 - ('floor', 'floor', 1, 0, 2)->('floor', 'floor', 2, 0, 2)
CustomRule call
('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [0]
result: 24 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
try: 22 - ('floor', 'floor', 0, 0, 2)->('grip1', 'floor', 0, 0, 2)
transition motion tried: False
CustomRule call
('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [0]
result: 16 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
try: 22 - ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 0, 1, 2)
CustomRule call
transition motion tried: True
('floor', 'floor', 0, 0, 2)->('floor', 'floor', 0, 1, 2) , diff: [3]
transition motion tried: False
CustomRule call
Rule for SweepLineTask
('floor', 'floor', 0, 0, 

result: 9 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
branching: 22->31 (5.85/100.0 s, steps/err: 45(225.285053253 ms)/0.0010487946727)
Rule for SweepLineTask
('floor', 'floor', 0, 0, 2)->('floor', 'grip1', 0, 0, 2) , diff: [1]
CustomRule call
result: 22 - ('floor', 'floor', 0, 0, 2)->('floor', 'grip1', 0, 0, 2) = fail
try: 27 - ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 1, 0, 2)
try: 31 - ('floor', 'floor', 1, 0, 2)->('floor', 'floor', 2, 0, 2)
result: 4 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
('grip1', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2) , diff: [0]
try: 22 - ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 0, 1, 2)
try: 26 - ('grip1', 'floor', 0, 0, 2)->('grip1', 'floor', 0, 1, 2)
result: 29 - ('grip1', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2) = fail
try: 22 - ('floor', 'floor', 0, 0, 2)->('floor', 'grip1', 0, 0, 2)
============= try go home (indy0) ===================
constrained motion tried: False
constrained mo

...

joint motion tried: True
CustomRule call
('floor', 'grip1', 0, 0, 2)->('floor', 'grip1', 0, 0, 2) , diff: []
=============== try extend to goal ('floor', 'grip1', 0, 0, 2) -> ('floor', 'grip1', 0, 1, 2) =================
result: 45 - ('floor', 'grip1', 0, 0, 2)->('floor', 'grip1', 0, 0, 2) = success
branching: 45->53 (8.17/100.0 s, steps/err: 13(211.744070053 ms)/5.94205328995e-16)
try: 53 - ('floor', 'grip1', 0, 0, 2)->('floor', 'grip1', 0, 1, 2)
joint motion tried: True
CustomRule call
('floor', 'floor', 2, 0, 2)->('floor', 'floor', 2, 0, 2) , diff: []
result: 52 - ('floor', 'floor', 2, 0, 2)->('floor', 'floor', 2, 0, 2) = success
branching: 52->54 (8.27/100.0 s, steps/err: 12(152.362108231 ms)/4.55245861612e-16)
try: 54 - ('floor', 'floor', 2, 0, 2)->('floor', 'grip1', 2, 0, 2)
CustomRule call
('floor', 'floor', 2, 0, 2)->('floor', 'grip1', 2, 0, 2) , diff: [1]
transition motion tried: False
CustomRule call
result: 54 - ('floor', 'floor', 2, 0, 2)->('floor', 'grip1', 2, 0, 2) = fail

CustomRule call
result: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'floor', 0, 2, 2) = fail
result: 44 - ('grip1', 'floor', 1, 0, 2)->('grip1', 'floor', 2, 0, 2) = success
branching: 55->60 (9.4/100.0 s, steps/err: 19(101.800918579 ms)/0.00153688698383)
branching: 44->61 (9.42/100.0 s, steps/err: 55(2356.75287247 ms)/0.00140657096198)
('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2) , diff: [2]
try: 60 - ('floor', 'grip1', 1, 2, 2)->('floor', 'grip1', 2, 2, 2)
Rule for SweepLineTask
Reserve next wp from 62
try: 54 - ('floor', 'floor', 2, 0, 2)->('floor', 'floor', 2, 1, 2)
try: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'floor', 0, 2, 2)
Removing 62 from [62] as next wp is reserved
result: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2) = success
CustomRule call
CustomRule call
branching: 55->62 (9.49/100.0 s, steps/err: 23(250.470161438 ms)/0.00152603904355)
('floor', 'grip1', 0, 2, 2)->('floor', 'floor', 0, 2, 2) , diff: [1]
('floor', 'floor', 2, 0, 2)->('floor

try: 64 - ('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2)
joint motion tried: True
CustomRule call
('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 0, 2, 2) , diff: []
=============== try extend to goal ('floor', 'grip1', 0, 2, 2) -> ('floor', 'grip1', 1, 2, 2) =================
result: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 0, 2, 2) = success
branching: 55->65 (12.22/100.0 s, steps/err: 11(125.608205795 ms)/5.52847587942e-16)
try: 65 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2)
transition motion tried: False
CustomRule call
('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) , diff: [2]
Rule for SweepLineTask
result: 64 - ('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) = fail
try: 64 - ('floor', 'floor', 0, 2, 2)->('grip1', 'floor', 0, 2, 2)
transition motion tried: False
CustomRule call
CustomRule call
('floor', 'floor', 0, 2, 2)->('grip1', 'floor', 0, 2, 2) , diff: [0]
('floor', 'floor', 0, 2, 2)->('grip1', 'floor', 0, 2, 2) , diff: 

try: 72 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 0, 2, 2)
try: 71 - ('grip1', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2)
branching: 64->72 (13.41/100.0 s, steps/err: 43(1372.68900871 ms)/0.00156803827659)
CustomRule call
('grip1', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2) , diff: [0]
try: 65 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2)
result: 71 - ('grip1', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2) = fail
============= try go home (indy0) ===================
try: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 0, 2, 2)
transition motion tried: True
CustomRule call
transition motion tried: False
('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2) , diff: [2]
CustomRule call
Rule for SweepLineTask
('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) , diff: [2]
Reserve next wp from 73
Removing 73 from [73] as next wp is reserved
Rule for SweepLineTask
result: 64 - ('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) = fail
result: 6

Removing 78 from [78] as next wp is reserved
result: 64 - ('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) = fail
try: 79 - ('floor', 'floor', 1, 0, 2)->('floor', 'floor', 2, 0, 2)
('grip1', 'floor', 0, 0, 2)->('grip1', 'floor', 0, 1, 2) , diff: [3]
transition motion tried: True
result: 55 - ('floor', 'grip1', 0, 2, 2)->('floor', 'grip1', 1, 2, 2) = success
try: 78 - ('floor', 'grip1', 1, 2, 2)->('floor', 'grip1', 2, 2, 2)
Rule for SweepLineTask
result: 29 - ('grip1', 'floor', 0, 0, 2)->('grip1', 'floor', 0, 1, 2) = fail
CustomRule call
('floor', 'floor', 0, 0, 2)->('floor', 'floor', 1, 0, 2) , diff: [2]
branching: 55->78 (14.43/100.0 s, steps/err: 13(198.682069778 ms)/0.00198922408959)
Rule for SweepLineTask
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0)
try: 67 - ('grip1', 'floor', 0, 2, 2)->('grip1', 'floor', 1, 2, 2)
transition motion tried: False
CustomRule call
Reserve next wp from 80
CustomRule call
('grip1', 'floor', 0, 2, 2)->('grip1', 'floor', 1, 2,

In [31]:
print(gtimer)

plan: 	49114.0 ms/1 = 49114.106 ms (49114.106/49114.106)



## play searched plan

In [32]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [33]:
snode_schedule = ppline.add_return_motion(snode_schedule)

joint motion tried: True
joint motion tried: True


In [34]:
ppline.play_schedule(snode_schedule, period=0.03)

In [ ]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}

## extend preserving goal-matching items

In [25]:
print(gtimer)

plan: 	207291.0 ms/10 = 20729.062 ms (9041.162/45383.357)



## extend only no reservation

In [25]:
print(gtimer)

plan: 	270954.0 ms/10 = 27095.38 ms (6255.591/68399.08)



## no extension

In [25]:
print(gtimer)

plan: 	226519.0 ms/10 = 22651.91 ms (8120.631/69353.952)



## extend_toward goal

In [30]:
print(gtimer)

plan: 	204994.0 ms/10 = 20499.448 ms (8105.979/60408.919)

